In [1]:
import pandas as pd
import psycopg2 as pg
import numpy as np

In [27]:
df = pd.read_csv("C:/Users/mamad/Downloads/Compressed/olist_order_items_dataset.csv")

In [45]:
df[:10000]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
9995,16f391bc8fc37407de41720cd92f6266,1,78b7b1ff2d3f06a589354ddf2f4f9db3,620c87c171fb2a6dd6e8bb4dec959fc6,2017-04-13 12:32:07,292.90,16.22
9996,16f4331c56f6c5c76a5ea85c7919f087,1,19c91ef95d509ea33eda93495c4d3481,06a2c3af7b3aee5d69171b0e14f0ee87,2018-07-02 21:27:05,122.99,36.63
9997,16f4a05a36f470dbed1f1c2ca6e2a616,1,0aabfb375647d9738ad0f7b4ea3653b1,37515688008a7a40ac93e3b2e4ab203f,2017-10-06 11:49:24,19.90,15.10
9998,16f4c47c722704ef26d0f086cb75d213,1,aca2eb7d00ea1a7b8ebd4e68314663af,955fee9216a65b617aa5c0531780ce60,2018-01-11 22:48:11,69.90,24.94


In [29]:
# set up configuration
host = "localhost"
port = "5432"
database = "postgres"
user = "postgres"
password = "kagome235"
setting = "dbname=" + database + " user=" + user + " host=" + host + " port=" + port + " password=" + password
engine = pg.connect(setting)

In [48]:
def insert_into_table(conn, df, table):
    """
    Using cursor.mogrify() to build the bulk insert query
    then cursor.execute() to execute the query
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]

    #     print(tuples)
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))

    # SQL quert to execute
    cursor = conn.cursor()
    values = [cursor.mogrify("(%s,%s,%s,%s,%s,%s,%s)", tup).decode('utf8') for tup in tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, pg.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("INSERT DONE")
    cursor.close()

In [49]:
insert_into_table(engine, df[:10000], 'order_items_dataset')

INSERT DONE
